In [257]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [258]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [259]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [260]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [261]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
market_data_scaled = StandardScaler().fit_transform(df_market_data[["price_change_percentage_24h","price_change_percentage_7d",	"price_change_percentage_14d",	"price_change_percentage_30d",	"price_change_percentage_60d",	"price_change_percentage_200d",	"price_change_percentage_1y"]])

In [262]:
# Create a DataFrame with the scaled data
df_market_transformed = pd.DataFrame(market_data_scaled, columns=["24hr", "7d", "14d", "30d", "60d", "200d", "1y"])

# Copy the crypto names from the original data
df_market_transformed["coin_id"] = df_market_data.index

# Set the coinid column as index
df_market_transformed = df_market_transformed.set_index("coin_id") 

# Display sample data
df_market_transformed.head()

,24hr,7d,14d,30d,60d,200d,1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Data.

In [263]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1,11))

In [264]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list

for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(market_data_scaled)
    inertia.append(k_model.inertia_)


In [265]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k, "inertia": inertia}

# Create a DataFrame using the elbow_data Dictionary
df_elbow = pd.DataFrame(elbow_data)

# Review the DataFrame
df_elbow.head()

,k,inertia
0,1,287.000000
1,2,212.123342
2,3,165.136752
3,4,79.022435
4,5,66.413051


In [266]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
df_elbow.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve",
    xticks=k
)

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 4

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [267]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=1)

In [268]:
# Fit the K-Means model using the scaled data
model.fit(df_market_transformed)

KMeans(n_clusters=4, random_state=1)

In [269]:
# Predict the clusters to group the cryptocurrencies using the scaled data
k_lower = model.predict(df_market_transformed)

# Print the resulting array of cluster values.
k_lower

array([2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 0, 0, 1, 0, 0, 0, 0],
      dtype=int32)

In [270]:
# Create a copy of the DataFrame
df_market_predictions = df_market_transformed.copy()

In [271]:
# Add a new column to the DataFrame with the predicted clusters
df_market_predictions['clusters_lower'] = k_lower

# Display sample data
df_market_predictions.head()

,24hr,7d,14d,30d,60d,200d,1y,clusters_lower
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,2
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,2
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,0
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,0
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,2


In [272]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
df_market_predictions.hvplot.scatter(
    x="24hr",
    y="7d",
    by="clusters_lower"
).opts(yformatter="%.0f")

:NdOverlay   [clusters_lower]
   :Scatter   [24hr]   (7d)

---

### Optimize Clusters with Principal Component Analysis.

In [273]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [274]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
info_pca = pca.fit_transform(df_market_predictions)
# View the first five rows of the DataFrame. 
info_pca[:5]

array([[ 0.44890795, -1.24537573, -0.8506404 ],
       [ 0.49536709, -0.899823  , -1.31755943],
       [-0.81884571,  0.07189909,  0.69501507],
       [-0.84035711,  0.08005407,  0.54435952],
       [ 0.81324004, -2.66952035, -1.64332113]])

In [275]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
pca.explained_variance_ratio_

array([0.34871677, 0.31363391, 0.22627118])

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 34.9% + 31.4% + 22.6% = 88.9% which is great!!

In [276]:
# Create a new DataFrame with the PCA data.
pca_df = pd.DataFrame(
    info_pca,
    columns=["PCA1", "PCA2", "PCA3"]
)

# Copy the crypto names from the original data
pca_df["coin_id"] = df_market_transformed.index

# Set the coinid column as index
pca_df = pca_df.set_index("coin_id") 

# Display sample data
pca_df.head()

,PCA1,PCA2,PCA3
coin_id,,,
bitcoin,0.448908,-1.245376,-0.850640
ethereum,0.495367,-0.899823,-1.317559
tether,-0.818846,0.071899,0.695015
ripple,-0.840357,0.080054,0.544360
bitcoin-cash,0.813240,-2.669520,-1.643321


---

### Find the Best Value for k Using the PCA Data

In [277]:
# Create a list with the number of k-values from 1 to 11
k_pca = list(range(1, 11))

In [278]:
# Create an empty list to store the inertia values
inertia_pca = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list

for i_pca in k_pca:
    k_model = KMeans(n_clusters=i_pca, random_state=1)
    k_model.fit(pca_df)
    inertia_pca.append(k_model.inertia_)



In [279]:
# Create a dictionary with the data to plot the Elbow curve
pca_elbow_data = {"k": k_pca, "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow_pca = pd.DataFrame(pca_elbow_data)
df_elbow_pca.head()

,k,inertia
0,1,287.000000
1,2,212.123342
2,3,165.136752
3,4,79.022435
4,5,66.413051


In [280]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
df_elbow_pca.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)


:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 3


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** Yes, K value seemed like we should use 4 based off inertia 

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [281]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=3, random_state=1)

In [282]:
# Fit the K-Means model using the PCA data
model.fit(pca_df)

KMeans(n_clusters=3, random_state=1)

In [283]:
# Predict the clusters to group the cryptocurrencies using the PCA data
k_3 = model.predict(pca_df)

# Print the resulting array of cluster values.
k_3

array([1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 2, 1, 0, 0, 2, 0, 0, 0, 0],
      dtype=int32)

In [284]:
# Create a copy of the DataFrame with the PCA data
pca_predictions_df = pca_df.copy()

# Add a new column to the DataFrame with the predicted clusters
pca_predictions_df["clusters"] = k_3

# Display sample data
pca_predictions_df.head()

,PCA1,PCA2,PCA3,clusters
coin_id,,,,
bitcoin,0.448908,-1.245376,-0.850640,1
ethereum,0.495367,-0.899823,-1.317559,1
tether,-0.818846,0.071899,0.695015,0
ripple,-0.840357,0.080054,0.544360,0
bitcoin-cash,0.813240,-2.669520,-1.643321,1


In [285]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
pca_predictions_df.hvplot.scatter(
    x="PCA1",
    y="PCA2",
    by="clusters"
)


:NdOverlay   [clusters]
   :Scatter   [PCA1]   (PCA2)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [286]:
# Composite plot to contrast the Elbow curves
df_elbow.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve",
    xticks=k
)

:Curve   [k]   (inertia)

In [294]:
df_elbow_pca.hvplot.line(
    x="k", 
    y="inertia", 
    title="PCA Elbow Curve", 
    xticks=k
)

:Curve   [k]   (inertia)

In [293]:
# Composite plot to contrast the clusters
df_market_predictions.hvplot.scatter(
    x="24hr",
    y="7d",
    by="clusters_lower",
    title= "KMeans"
).opts(yformatter="%.0f")

:NdOverlay   [clusters_lower]
   :Scatter   [24hr]   (7d)

In [292]:
pca_predictions_df.hvplot.scatter(
    x="PCA1",
    y="PCA2",
    by="clusters",
    title="PCA"
)

:NdOverlay   [clusters]
   :Scatter   [PCA1]   (PCA2)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** The three cluster groups definitely created more clusters in which are closer together, rather than the group of 4 clusters having 2 dots that were just a separate color but by themselves. 